# Development environment setup

## Libraries

In [55]:
from pathlib import Path

import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from pprint import pprint

## Paths

In [2]:
project_folder = Path(".").absolute().parent


## Plotly preferences

In [3]:
# Change the default theme to "plotly_white"
pio.templates.default = "plotly_white"


# Datasets

## Import raw data

### Customers

In [4]:
customers = pd.read_csv(
    project_folder.joinpath("data", "raw", "olist_customers_dataset.csv")
)
customers.head(2)


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP


### Geolocation

In [5]:
geolocation = pd.read_csv(
    project_folder.joinpath("data", "raw", "olist_geolocation_dataset.csv")
)
geolocation.head(2)


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP


### Orders

In [6]:
orders = pd.read_csv(project_folder.joinpath("data", "raw", "olist_orders_dataset.csv"))

orders.head(2)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00


### Order Items

In [7]:
order_items = pd.read_csv(
    project_folder.joinpath("data", "raw", "olist_order_items_dataset.csv")
)
order_items.head(2)


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93


### Order Payments

In [8]:
order_payments = pd.read_csv(
    project_folder.joinpath("data", "raw", "olist_order_payments_dataset.csv")
)

order_payments.head(2)


,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39


### Order Reviews

In [9]:
order_reviews = pd.read_csv(
    project_folder.joinpath("data", "raw", "olist_order_reviews_dataset.csv")
)

order_reviews.head(2)


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13


### Products

In [10]:
products = pd.read_csv(
    project_folder.joinpath("data", "raw", "olist_products_dataset.csv")
)

products.head(2)


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0


### Sellers

In [11]:
sellers = pd.read_csv(
    project_folder.joinpath("data", "raw", "olist_sellers_dataset.csv")
)

sellers.head(2)


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP


### Product Category Name Translation

In [12]:
product_category_name_translation = pd.read_csv(
    project_folder.joinpath("data", "raw", "product_category_name_translation.csv")
)

product_category_name_translation.head(2)


,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories


# Exploratory Data Analysis (EDA)

## Number of orders and payment values by state and order status

In [13]:
# Creates a aggregated dataframe with the number of orders and the total payment value by state and order status
value_orders_by_state_status = (
    orders.merge(customers, how="left", on="customer_id")
    .merge(order_payments, how="left", on="order_id")
    .loc[:, ["customer_state", "order_id", "order_status", "payment_value"]]
    .groupby(by=["customer_state", "order_status"])
    .agg({"order_id": "nunique", "payment_value": "sum"})
    .reset_index()
    .rename(columns={"order_id": "n_orders"})
    .sort_values(by="payment_value", ascending=False)
)

value_orders_by_state_status


,customer_state,order_status,n_orders,payment_value
139,SP,delivered,40501,5770266.19
101,RJ,delivered,12350,2055690.45
51,MG,delivered,11354,1819277.61
120,RS,delivered,5345,861802.40
94,PR,delivered,4923,781919.55
...,...,...,...,...
7,AL,unavailable,1,69.38
131,SE,canceled,1,61.79
49,MG,approved,1,61.62
133,SE,invoiced,1,43.05


In [78]:
pivot_value_by_state_status = (
    value_orders_by_state_status.pivot_table(
        index="customer_state", columns="order_status", values="payment_value"
    )
    .fillna(0)
    .reset_index()
    .rename_axis(None, axis=1)
    .sort_values(by="customer_state", ascending=False)
)

pivot_value_by_state_status.head(2)


,customer_state,approved,canceled,created,delivered,invoiced,processing,shipped,unavailable
26,TO,0.00,250.57,0.00,60007.37,0.00,96.98,999.50,130.91
25,SP,179.46,55829.85,160.94,5770266.19,30663.24,31436.74,46229.42,63461.12


In [77]:
pivot_orders_by_state_status = (
    value_orders_by_state_status.pivot_table(
        index="customer_state", columns="order_status", values="n_orders"
    )
    .fillna(0)
    .reset_index()
    .rename_axis(None, axis=1)
    .sort_values(by="customer_state", ascending=False)
)

pivot_orders_by_state_status.head(2)


,customer_state,approved,canceled,created,delivered,invoiced,processing,shipped,unavailable
26,TO,0.0,1.0,0.0,274.0,0.0,1.0,3.0,1.0
25,SP,1.0,327.0,1.0,40501.0,160.0,136.0,328.0,292.0


In [97]:
map_status_color = {
    "approved": "#6e35ff",
    "canceled": "#ff4d65",
    "created": "#6e35ff",
    "delivered": "#00c456",
    "invoiced": "#6e35ff",
    "processing": "#6e35ff",
    "shipped": "#6e35ff",
    "unavailable": "#ff4d65",
}

pprint(map_status_color)

{'approved': '#6e35ff',
 'canceled': '#ff4d65',
 'created': '#6e35ff',
 'delivered': '#00c456',
 'invoiced': '#6e35ff',
 'processing': '#6e35ff',
 'shipped': '#6e35ff',
 'unavailable': '#ff4d65'}


In [98]:
status_order = [
    "created",
    "processing",
    "approved",
    "invoiced",
    "shipped",
    "delivered",
    "unavailable",
    "canceled",
]

pprint(status_order)

['created',
 'processing',
 'approved',
 'invoiced',
 'shipped',
 'delivered',
 'unavailable',
 'canceled']


In [96]:
# Creates a plotly figure with the number of orders and the total payment value by state and order status
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Total Payment Value", "Number of Orders"),
    shared_yaxes=True,
    specs=[[{"type": "bar"}, {"type": "bar"}]],
)

for status in status_order:
    trace1 = go.Bar(
        y=pivot_value_by_state_status["customer_state"],
        x=pivot_value_by_state_status[status],
        orientation="h",
        name=status.title(),
        marker=dict(color=map_status_color[status]),
    )

    trace2 = go.Bar(
        y=pivot_orders_by_state_status["customer_state"],
        x=pivot_orders_by_state_status[status],
        orientation="h",
        name=status.title(),
        showlegend=False,
        marker=dict(color=map_status_color[status]),
    )

    fig.add_trace(trace1, row=1, col=1)
    fig.add_trace(trace2, row=1, col=2)

fig.update_layout(
    title_text="Number of Total Payment Value and Orders by State and Order Status",
    height=600,
    width=1200,
)

fig.update_layout(
    barmode="stack",
    xaxis=dict(title="Value (BRL)"),
    xaxis2=dict(title="# Orders"),
)

fig.show()
